In [52]:
%load_ext autoreload
%autoreload 2

from HSM_model_copy import HouseholdSpecializationModelClass
import numpy as np
import pandas as pd
from tqdm import tqdm
from matplotlib import pyplot as plt
from scipy import optimize


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


**Question 1**

In [53]:
model = HouseholdSpecializationModelClass()

print(model.par)

namespace(rho=2.0, nu=0.001, epsilon=1.0, omega=0.5, alpha=0.5, sigma=1.0, wM=1.0, wF=1.0, wF_vec=array([0.8, 0.9, 1. , 1.1, 1.2]), beta0_target=0.4, beta1_target=-0.1)


In [ ]:
q1_sol = {}
for i in ['HF', 'HM', 'alpha', 'sigma']:
    q1_sol[i] = []

for model.par.alpha in [0.25, 0.50, 0.75]:
    for model.par.sigma in [0.5, 1.0, 1.5]:
        opt = model.solve_discrete()
        q1_sol['HF'].append(opt.HF)
        q1_sol['HM'].append(opt.HM)
        q1_sol['alpha'].append(model.par.alpha)
        q1_sol['sigma'].append(model.par.sigma)


In [ ]:
q1_sol = pd.DataFrame(q1_sol)
q1_sol['HF/HM'] = q1_sol['HF']/q1_sol['HM']
q1_sol

**Plot Q1 solutions**

In [ ]:
%matplotlib inline
fig = plt.figure(constrained_layout=True)
ax = fig.add_subplot(1,1,1,projection='3d') # create a 3d type axis 
x = q1_sol['alpha']
y = q1_sol['sigma']
hf_hm = q1_sol['HF/HM']
ax.set_xlabel('alpha')
ax.set_ylabel('sigma')
ax.set_zlabel('HF/HM')
ax.invert_xaxis()
ax.plot_trisurf(x,y, hf_hm, cmap='PiYG'); # create surface plot in the axis
# note: fig.add_subplot(a,b,c) creates the c'th subplot in a grid of a times b plots

**Question 2**


In [ ]:
discrete_solution = model.solve_wF_vec(discrete=True)
df_discrete = {}
df_discrete['HF'] = discrete_solution.HF_vec
df_discrete['HM'] = discrete_solution.HM_vec
df_discrete['LF'] = discrete_solution.LF_vec
df_discrete['LM'] = discrete_solution.LM_vec


df_discrete = pd.DataFrame(df_discrete)
df_discrete['utility'] = model.calc_utility(df_discrete.LM, df_discrete.HM, df_discrete.LF, df_discrete.HF)
df_discrete['logHF/logHM'] = np.log(df_discrete['HF']) - np.log(df_discrete['HM'])
df_discrete['WF'] =  model.par.wF_vec.tolist()
df_discrete['WM'] = model.par.wM
df_discrete['logWF/logWM'] = np.log(df_discrete['WF']) - np.log(df_discrete['WM'])

df_discrete


In [ ]:
x = df_discrete['logWF/logWM']
y = df_discrete['logHF/logHM']

fig = plt.figure(constrained_layout=True)

ax = fig.add_subplot(1,1,1)
ax.grid()
ax.set_ylabel('$\log (HF)/\log (HM)$')
ax.set_xlabel('$\log (WF)/\log (WM)$')
ax.scatter(x, y)

**Question 3**

In [ ]:
solutions = {}
for i in ['HF', 'HM', 'LF', 'LM', 'WF', 'WM', 'Utility']:
    solutions[i] = []


for i in [0.8, 0.9, 1.0, 1.1, 1.2]:
    model.par.wF = i
    solution = model.solve()
    solutions['HF'].append(solution.HF)
    solutions['LF'].append(solution.LF)
    solutions['HM'].append(solution.HM)
    solutions['LM'].append(solution.LM)
    solutions['WF'].append(model.par.wF)
    solutions['WM'].append(model.par.wM)
    solutions['Utility'].append(solution.U)
    
#
df = pd.DataFrame(solutions)


***Solving the problem using the function with the defined wf_VEC***

In [ ]:
### solve model using WF_vec gives us the same result as running the loop above 
contionoussolution = model.solve_wF_vec(discrete=False)
df1 = {}
df1['HF'] = contionoussolution.HF_vec
df1['HM'] = contionoussolution.HM_vec
df1['LF'] = contionoussolution.LF_vec
df1['LM'] = contionoussolution.LM_vec

df1 = pd.DataFrame(df1)
df1['utility'] = model.calc_utility(df1.LM, df1.HM, df1.LF, df1.HF)

df1


In [ ]:
df['logHF/logHM'] = np.log(df['HF']) - np.log(df['HM'])

df['logWF/logWM'] = np.log(df['WF']) - np.log(df['WM'])

df['alternative_utility'] = model.calc_utility(df.LM, df.HM, df.LF, df.HF)

df

**Den her graf bliver flot hvis man laver den med nelder-mead i stedet for slsqp**

In [ ]:
from matplotlib import pyplot as plt
fig = plt.figure()
ax = fig.add_subplot(1,1,1)
ax.scatter(df['logWF/logWM'], df['logHF/logHM'])
ax.grid()
ax.set_ylabel('$\log (HF)/\log (HM)$')
ax.set_xlabel('$\log (WF)/\log (WM)$')
ax.plot()

In [ ]:
print(model.solve())

**Run regression**

$\log \frac{H_F}{H_M} = \beta_0 + \beta_1 \log \frac{w_F}{w_M} $

In [ ]:
model.run_regression()
print(f'beta0 = {model.sol.beta0:.3} and beta_1 = {model.sol.beta1:.3}')

**Estimate $\sigma$ and $\alpha$ such that $(\beta_{0}-\hat \beta_0)^2+(\beta_1-\hat \beta_1)^2$ is minimized, where we assume that $\beta_0=0.4$ and $\beta_1 \approx -0.1$**



In [ ]:

beta0 = 0.4
beta1 = -0.1
def lossfunction(beta0hat, beta1hat, beta0=0.4, beta1=-0.1):
    return (beta0-beta0hat)**2 + (beta1-beta1hat)**2 

winners = {}
winners['alpha'] = []
winners['sigma'] = []
winners['loss'] = []

for model.par.alpha in tqdm(np.linspace(0,1, 50)):
    for model.par.sigma in np.linspace(0, 3, 50):
            sol = model.solve_wF_vec()
            model.run_regression()
            beta0hat = model.sol.beta0
            beta1hat = model.sol.beta1
            #print(f'beta0 = {model.sol.beta0:.3} and beta_1 = {model.sol.beta1:.3}')
            loss = lossfunction(beta0hat, beta1hat)
            if -0.4 <= loss <= 0.4:
                 winners['alpha'].append(model.par.alpha)
                 winners['sigma'].append(model.par.sigma)
                 winners['loss'].append(loss)



In [ ]:
winner_df = pd.DataFrame(winners)
winner_df.sort_values(by='loss')

**Estimate like Jeppe Did in the ASAD lecture**

In [67]:
def obj(x, parnames, do_print=False):
      # a. update parameters
    for xval,parname in zip(x,parnames):
        par.__dict__[parname] = xval
        if do_print: print(f'{parname:10s} = {xval:.4f}')

    if do_print: print('')

    # b. simulate and calculate moments
    model.solve_wF_vec()
    model.run_regression()

    # c. compare with data
    error = model.loss_function()

    return error



**Calculate objective at initial values:**

In [82]:
par = model.par
model.par.alpha = 0.5
model.par.sigma = 0.8
parnames = ['alpha','sigma']
x0 = [par.__dict__[parname] for parname in parnames]

error_ = obj(x0,parnames,do_print=True)
print(error_)

alpha      = 0.5000
sigma      = 0.8000

0.4574755414276454


In [85]:
bounds = ((0,1),(0,None))
res = optimize.minimize(obj,x0,bounds=bounds,method='nelder-mead',args=(parnames))

In [86]:
assert res.success

In [87]:
error_ = obj(res.x, parnames, do_print=True)
print(error_)

alpha      = 0.9996
sigma      = 0.0527

0.0018473935346442572


**Estimate with module in "HSM_model_copy.py"**

In [88]:
print(model.par.alpha, model.par.sigma)


0.9995533971148152 0.05266963043670445


In [89]:
error = model.estimate()

alpha      = 0.9882
sigma      = 0.0910



In [90]:
print(error)

6.2382201302892e-08
